In [2]:
%%bash
set -e

sudo apt-get update
sudo apt-get install -y wget apt-transport-https gnupg lsb-release

wget -qO - https://aquasecurity.github.io/trivy-repo/deb/public.key | sudo apt-key add -
echo "deb https://aquasecurity.github.io/trivy-repo/deb $(lsb_release -sc) main" \
  | sudo tee /etc/apt/sources.list.d/trivy.list

sudo apt-get update
sudo apt-get install -y trivy

trivy --version


Hit:1 https://packages.cloud.google.com/apt gcsfuse-bullseye InRelease
Hit:2 https://download.docker.com/linux/debian bullseye InRelease
Hit:3 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:4 https://deb.debian.org/debian bullseye InRelease
Hit:5 https://packages.cloud.google.com/apt google-compute-engine-bullseye-stable InRelease
Get:6 https://deb.debian.org/debian-security bullseye-security InRelease [27.2 kB]
Hit:7 https://deb.debian.org/debian bullseye-updates InRelease
Hit:8 https://packages.cloud.google.com/apt cloud-sdk-bullseye InRelease
Hit:9 https://packages.cloud.google.com/apt google-fast-socket InRelease
Fetched 27.2 kB in 1s (21.8 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
apt-transport-https is already the newest version (2.2.4).
gnupg is already the newest version (2.2.27-2+deb11u2).
gnupg set to manually installed.
lsb-release is already the newest version (11.1.0).
lsb-

OK
deb https://aquasecurity.github.io/trivy-repo/deb bullseye main
Hit:1 https://download.docker.com/linux/debian bullseye InRelease
Hit:2 https://deb.debian.org/debian bullseye InRelease
Hit:3 https://packages.cloud.google.com/apt gcsfuse-bullseye InRelease
Hit:4 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:5 https://deb.debian.org/debian-security bullseye-security InRelease
Hit:6 https://deb.debian.org/debian bullseye-updates InRelease
Hit:7 https://packages.cloud.google.com/apt google-compute-engine-bullseye-stable InRelease
Get:8 https://aquasecurity.github.io/trivy-repo/deb bullseye InRelease [3064 B]
Hit:9 https://packages.cloud.google.com/apt cloud-sdk-bullseye InRelease
Hit:10 https://packages.cloud.google.com/apt google-fast-socket InRelease
Get:11 https://aquasecurity.github.io/trivy-repo/deb bullseye/main amd64 Packages [370 B]
Fetched 3434 B in 1s (2512 B/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading

dpkg-preconfigure: unable to re-open stdin: No such file or directory


Fetched 39.3 MB in 1s (34.0 MB/s)
Selecting previously unselected package trivy.
(Reading database ... 145494 files and directories currently installed.)
Preparing to unpack .../trivy_0.54.1_amd64.deb ...
Unpacking trivy (0.54.1) ...
Setting up trivy (0.54.1) ...
Version: 0.54.1


In [5]:
import json
import subprocess
import shutil
import requests
from google.auth import default
from google.auth.transport.requests import Request

# =====================================================
# CONFIGURATION (Notebook-safe)
# =====================================================
IMAGE = "nginx:latest"          # <-- change image here
LOCATION = "us-central1"        # Vertex AI region

# =====================================================
# PRE-FLIGHT CHECKS
# =====================================================
print("🔎 Performing pre-flight checks...")

if not shutil.which("trivy"):
    raise RuntimeError(
        "❌ Trivy is not installed. Install Trivy on the Workbench VM first."
    )

# =====================================================
# RUN TRIVY SCAN
# =====================================================
print(f"🐳 Scanning image: {IMAGE}")

subprocess.run(
    [
        "trivy", "image",
        "--scanners", "vuln",
        "--severity", "CRITICAL,HIGH,MEDIUM",
        "--format", "json",
        "--output", "trivy-report.json",
        IMAGE,
    ],
    check=True,
)

# =====================================================
# PARSE TRIVY RESULTS
# =====================================================
with open("trivy-report.json") as f:
    report = json.load(f)

def count_vulns(severity):
    return sum(
        1
        for result in report.get("Results", [])
        for vuln in (result.get("Vulnerabilities") or [])
        if vuln.get("Severity") == severity
    )

critical = count_vulns("CRITICAL")
high = count_vulns("HIGH")
medium = count_vulns("MEDIUM")

print(f"📊 Vulnerability Summary → CRITICAL={critical}, HIGH={high}, MEDIUM={medium}")

# =====================================================
# PROMPT ENGINEERING
# =====================================================
PROMPT = f"""
You are a Senior DevSecOps Security Reviewer approving container images
for production Kubernetes deployments.

Scan Results:
Image: {IMAGE}
Critical vulnerabilities: {critical}
High vulnerabilities: {high}
Medium vulnerabilities: {medium}

Hard Policy:
- If Critical > 0 → DO_NOT_DEPLOY
- If High > 5 → DO_NOT_DEPLOY

Assumptions:
- Internet-facing workload
- No runtime security controls
- Production environment

Return ONLY valid JSON in the following schema:

{{
  "decision": "DEPLOY | DO_NOT_DEPLOY",
  "risk_level": "LOW | MEDIUM | HIGH | CRITICAL",
  "summary": "single line justification"
}}
"""

# =====================================================
# AUTHENTICATION (ADC)
# =====================================================
credentials, project_id = default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
credentials.refresh(Request())
access_token = credentials.token

print(f"🔐 Authenticated with project: {project_id}")

# =====================================================
# CALL GEMINI 2.5 FLASH VIA VERTEX AI
# =====================================================
endpoint = (
    f"https://{LOCATION}-aiplatform.googleapis.com/v1/"
    f"projects/{project_id}/locations/{LOCATION}/"
    f"publishers/google/models/gemini-2.5-flash:generateContent"
)

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
}

payload = {
    "contents": [
        {
            "role": "user",
            "parts": [{"text": PROMPT}],
        }
    ]
}

print("🤖 Calling Gemini 2.5 Flash via Vertex AI...")

response = requests.post(endpoint, headers=headers, json=payload)
response.raise_for_status()

data = response.json()

print("🔎 Raw Gemini response:")
print(json.dumps(data, indent=2))

# =====================================================
# PARSE & SANITIZE GEMINI RESPONSE
# =====================================================
try:
    raw_text = data["candidates"][0]["content"]["parts"][0]["text"]
except (KeyError, IndexError):
    raise RuntimeError("❌ Gemini returned an unexpected response structure")

print("🧠 Gemini raw text:")
print(raw_text)

# Remove Markdown code fences if present
clean_text = (
    raw_text
    .replace("```json", "")
    .replace("```", "")
    .strip()
)

try:
    decision_json = json.loads(clean_text)
except json.JSONDecodeError as e:
    raise RuntimeError(
        f"❌ Gemini returned invalid JSON after sanitization:\n{clean_text}"
    ) from e

# =====================================================
# EXTRACT DECISION
# =====================================================
decision = decision_json.get("decision")
risk = decision_json.get("risk_level")
summary = decision_json.get("summary")

if decision not in {"DEPLOY", "DO_NOT_DEPLOY"}:
    raise RuntimeError("❌ Invalid decision value returned by Gemini")

# =====================================================
# FINAL DECISION
# =====================================================
print("\n================ FINAL DECISION ================")
print(f"🧠 Decision : {decision}")
print(f"⚠️  Risk     : {risk}")
print(f"📝 Summary  : {summary}")
print("================================================")

if decision == "DEPLOY":
    print("✅ Image approved for deployment")
else:
    print("❌ Image rejected for deployment")


🔎 Performing pre-flight checks...
🐳 Scanning image: nginx:latest


2025-12-20T01:36:16Z	INFO	[vuln] Vulnerability scanning is enabled
2025-12-20T01:36:17Z	INFO	Detected OS	family="debian" version="13.2"
2025-12-20T01:36:17Z	INFO	[debian] Detecting vulnerabilities...	os_version="13" pkg_num=150
2025-12-20T01:36:17Z	INFO	Number of language-specific files	num=0
2025-12-20T01:36:17Z	WARN	Using severities from other vendors for some vulnerabilities. Read https://aquasecurity.github.io/trivy/v0.54/docs/scanner/vulnerability#severity-selection for details.


📊 Vulnerability Summary → CRITICAL=0, HIGH=4, MEDIUM=17
🔐 Authenticated with project: steam-aria-479903-s9
🤖 Calling Gemini 2.5 Flash via Vertex AI...
🔎 Raw Gemini response:
{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "```json\n{\n  \"decision\": \"DO_NOT_DEPLOY\",\n  \"risk_level\": \"HIGH\",\n  \"summary\": \"Image contains 4 high and 17 medium vulnerabilities; too risky for an internet-facing production environment lacking runtime security controls.\"\n}\n```"
          }
        ]
      },
      "finishReason": "STOP",
      "avgLogprobs": -11.649886592741936
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 155,
    "candidatesTokenCount": 62,
    "totalTokenCount": 1944,
    "trafficType": "ON_DEMAND",
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 155
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 62